<a href="https://colab.research.google.com/github/MichaelNeman/AQI_Summary/blob/main/TSS_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid, KFold, cross_val_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.tree import plot_tree
import plotly.express as px
import kaleido
import matplotlib.pyplot as plt
import pickle
from google.colab import files

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning: 


This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.

  from .kaleido import Kaleido


# <font color='green'>Import data</font>

In [ ]:
with open('X_TSS_top25_and_y.pkl', 'rb') as f:
    data = pickle.load(f)
X = data['X']
y= data['y']

In [ ]:
with open('X_TSS_top25_and_y_trans.pkl', 'rb') as f:
    data = pickle.load(f)
X_trans = data['X']
y_trans = data['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
#perform yeo-johnson transformation

pt = PowerTransformer(method='yeo-johnson')

# Fit on training data
X_train_t = pd.DataFrame(pt.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
y_train_t = pt.fit_transform(y_train)

# Transform test data
X_test_t = pd.DataFrame(pt.transform(X_test), columns=X_test.columns, index=X_test.index)
y_test_t = pt.transform(y_test)


# <font color='green'>Random Forest Regression</font>

### <font color='lightgreen'> f-score features - Disregard, using Recursive feature elimination.</font>

In [ ]:
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [2, 3],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20],
    'bootstrap': [True],
    'min_impurity_decrease': [0.0, 0.025, 0.05],

}

In [ ]:
rf = RandomForestRegressor(random_state = 42)

grid_search = GridSearchCV(
    estimator = rf,
    param_grid = param_grid,
    cv =  5,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    verbose = 2
)

grid_search.fit(X_train_f, y_train_f)

print("Best Parameters:", grid_search.best_params_)
print("Best Score (neg MSE)", grid_search.best_score_)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.025, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Score (neg MSE) -31837.99338382308


In [ ]:
y_pred_f = grid_search.predict(X_test_f)

In [ ]:
print("Test MSE:", mean_squared_error(y_test_f, y_pred_f))
print("Test R2:", r2_score(y_test_f, y_pred_f))

Test MSE: 22722.669512685854
Test R2: 0.5538712143913583


In [ ]:
y_f_plot = pd.DataFrame({'True Values': y_test_f, 'Predicted Values': y_pred_f})
fig = px.scatter(y_f_plot, x = 'True Values', y = 'Predicted Values')
fig.add_shape(type='line',
              x0 = y_f_plot['True Values'].min(), y0 = y_f_plot['True Values'].min(),
              x1 = y_f_plot['True Values'].max(), y1 = y_f_plot['True Values'].max(),
              line = dict(color='red', dash = 'dash'))
fig.show()

In [ ]:
y_f_plot2 = pd.DataFrame({
    'Index': range(len(y_test_f)),
    'True values': y_test_f,
    'Predicted values': y_pred_f
})

df_long = y_f_plot2.melt(id_vars='Index', value_vars=['True values', 'Predicted values'],
                       var_name='Type', value_name='Value')

fig = px.scatter(df_long, x='Index', y='Value', color='Type',
                 title='True vs Predicted Values')
fig.show()

### <font color='lightgreen'> Random Forest feature importance - Disregard, using Recursive feature elimination</font>

In [ ]:
rf = RandomForestRegressor(random_state = 42)

grid_search_rf = GridSearchCV(
    estimator = rf,
    param_grid = param_grid,
    cv =  5,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    verbose = 2
)

grid_search_rf.fit(X_train_rf, y_train_rf)

print("Best Parameters:", grid_search_rf.best_params_)
print("Best Score (neg MSE)", grid_search_rf.best_score_)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Score (neg MSE) -32143.500357661396


In [ ]:
y_pred_rf = grid_search_rf.predict(X_test_rf)

print("Test MSE:", mean_squared_error(y_test_f, y_pred_rf))
print("Test R2:", r2_score(y_test_f, y_pred_rf))

Test MSE: 24098.975339120432
Test R2: 0.526849316870446


In [ ]:
y_rf_plot = pd.DataFrame({'True Values': y_test_rf, 'Predicted Values': y_pred_rf})
fig = px.scatter(y_rf_plot, x = 'True Values', y = 'Predicted Values', trendline = 'ols')
fig.show()

In [ ]:
y_rf_plot2 = pd.DataFrame({
    'Index': range(len(y_test_f)),
    'True values': y_test_f,
    'Predicted values': y_pred_rf
})

df_long = y_rf_plot2.melt(id_vars='Index', value_vars=['True values', 'Predicted values'],
                       var_name='Type', value_name='Value')

fig = px.scatter(df_long, x='Index', y='Value', color='Type',
                 title='True vs Predicted Values')
fig.show()

### <font color='lightgreen'> Recursive Feature Elimination</font>

In [ ]:
rf = RandomForestRegressor(random_state = 42)

grid_search_RFE = GridSearchCV(
    estimator = rf,
    param_grid = param_grid,
    cv =  5,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    verbose = 2
)

grid_search_RFE.fit(X_train, y_train)

print("Best Parameters:", grid_search_RFE.best_params_)
print("Best Score (neg MSE)", grid_search_RFE.best_score_)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best Parameters: {'bootstrap': True, 'max_depth': 15, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Score (neg MSE) -0.46272482590895814


In [ ]:
rf_regr = RandomForestRegressor(bootstrap = True,
                                max_depth = 15,
                                #max_features = 'log2',
                                max_features = None,
                                max_leaf_nodes = None,
                                min_impurity_decrease = 0.0,
                                min_samples_leaf = 2,
                                min_samples_split = 2,
                                n_estimators = 100,
                                random_state = 42)

In [ ]:
TSS_rf = rf_regr.fit(X_train, y_train)

In [ ]:
y_pred_RF = TSS_rf.predict(X_test)

print("Test RMSE:", root_mean_squared_error(y_test, y_pred_RF))
print("Test MAE:", mean_absolute_error(y_test, y_pred_RF))

Test RMSE: 156.43358539598003
Test MAE: 82.68152342162945


In [ ]:
residuals = y_test - y_pred_RF

tss_res = pd.DataFrame({
    "Fitted": y_pred_RF,
    "Residuals": residuals
})

fig = px.scatter(tss_res, x="Fitted", y="Residuals",title="TSS Residuals vs. Fitted Values"
)

fig.add_hline(y=0)
fig.show()

In [ ]:
y_RFE_plot = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_pred_RF})
fig = px.scatter(y_RFE_plot, x = 'True Values', y = 'Predicted Values', title="Random Forest True vs Predicted TSS Values", trendline="ols")
fig.show()


In [ ]:
y_RFE_plot2 = pd.DataFrame({
    'Index': range(len(y_test)),
    'True values': y_test,
    'Predicted values': y_pred_RF
})

df_long = y_RFE_plot2.melt(id_vars='Index', value_vars=['True values', 'Predicted values'],
                       var_name='Type', value_name='Value')

fig = px.scatter(df_long, x='Index', y='Value', color='Type',
                 title='Random Forest True vs Predicted TSS Values')
fig.show()

In [ ]:
pd.DataFrame({'y_test': y_test, 'y_pred_RF': y_pred_RF}).to_csv('Random_Forest_Predictions.csv', index = False)

In [ ]:
import joblib

# Save the best estimator (model with best parameters)
joblib.dump(grid_search_RFE.best_estimator_, 'RF_best_model.pkl')

['RF_best_model.pkl']

# <font color='green'>Gradient Boosting Regression</font>

In [ ]:
param_grid = {
    'loss': ['squared_error'],
    'learning_rate' : [0.05, 0.1, 0.2],
    'n_estimators': [100, 150],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [2, 3],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20],
    'min_impurity_decrease': [0.0, 0.025, 0.05],

}

In [ ]:
gbr = GradientBoostingRegressor(random_state = 42)

grid_search_gbr_RFE = GridSearchCV(
    estimator = gbr,
    param_grid = param_grid,
    cv =  5,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    verbose = 2
)

grid_search_gbr_RFE.fit(X_train, y_train)

print("Best Parameters:", grid_search_gbr_RFE.best_params_)
print("Best Score (neg MSE)", grid_search_gbr_RFE.best_score_)

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits
Best Parameters: {'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.05, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Best Score (neg MSE) -31963.444813440048


In [ ]:
#train on full training data using best parameters

gbr_regr = GradientBoostingRegressor(
    learning_rate = 0.1,
    loss = "squared_error",
    max_depth = 10,
    max_features = 'log2',
    max_leaf_nodes = None,
    min_impurity_decrease = 0.05,
    min_samples_leaf = 2,
    min_samples_split = 2,
    n_estimators = 100,
    random_state = 42
)

TSS_gbr = gbr_regr.fit(X_train, y_train)

In [ ]:
y_pred_GBR = TSS_gbr.predict(X_test)

print("Test RMSE:", root_mean_squared_error(y_test, y_pred_GBR))
print("Test MAE:", mean_absolute_error(y_test, y_pred_GBR))

Test RMSE: 159.87075231790422
Test MAE: 81.77537066826751


In [ ]:
residuals_gbr = y_test - y_pred_GBR

tss_res_gbr = pd.DataFrame({
    "Fitted": y_pred_GBR,
    "Residuals": residuals
})

fig = px.scatter(tss_res, x="Fitted", y="Residuals",title="TSS GBR Residuals vs. Fitted Values"
)

fig.add_hline(y=0)
fig.show()

In [ ]:
y_RFE_plot = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_pred_GBR})
fig = px.scatter(y_RFE_plot, x = 'True Values', y = 'Predicted Values', title="GBR True vs Predicted TSS")
fig.show()

In [ ]:
y_RFE_plot2 = pd.DataFrame({
    'Index': range(len(y_test)),
    'True values': y_test,
    'Predicted values': y_pred_GBR
})

df_long = y_RFE_plot2.melt(id_vars='Index', value_vars=['True values', 'Predicted values'],
                       var_name='Type', value_name='Value')

fig = px.scatter(df_long, x='Index', y='Value', color='Type',
                 title='GBR True vs Predicted TSS Values')
fig.show()

In [ ]:
import joblib

# Save the best estimator (model with best parameters)
joblib.dump(grid_search_gbr_RFE.best_estimator_, 'GBR_best_model.pkl')

['GBR_best_model.pkl']

In [ ]:
pd.DataFrame({'y_test': y_test, 'y_pred_GBR': y_pred_GBR}).to_csv('Gradient_Boosted_Regression_Predictions.csv', index = False)

Import and plot BART results using Plotly so plot styling are consistent

In [ ]:
tss_bart = pd.read_csv('/content/tss_bart_results.csv')

In [ ]:
residuals_bart = tss_bart['TSS'] - tss_bart['.pred']

tss_res_bart = pd.DataFrame({
    "Fitted": tss_bart['.pred'],
    "Residuals": residuals_bart
})

fig = px.scatter(tss_res, x="Fitted", y="Residuals",title="TSS BART Residuals vs. Fitted Values"
)

fig.add_hline(y=0)
fig.show()

In [ ]:
bart_true_fitted = pd.DataFrame({'True Values': tss_bart['TSS'], 'Predicted Values': tss_bart['.pred']})
fig = px.scatter(y_RFE_plot, x = 'True Values', y = 'Predicted Values', title="BART TSS True vs Predicted TSS")
fig.show()

In [ ]:
tss_overlay = pd.DataFrame({
    'Index': range(len(tss_bart['TSS'])),
    'True values': tss_bart['TSS'],
    'Predicted values': tss_bart['.pred']
})

df_long = tss_overlay.melt(id_vars='Index', value_vars=['True values', 'Predicted values'],
                       var_name='Type', value_name='Value')

fig = px.scatter(df_long, x='Index', y='Value', color='Type',
                 title='BART True vs Predicted TSS Values')
fig.show()